# aggregate results

In [1]:
import numpy as np
import pandas as pd
import xgboost
from sklearn.metrics import roc_auc_score

/opt/conda/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_hdf("data-r1.h5", "train")
df2 = pd.read_hdf("data-r2.h5", "train")

In [3]:
m2 = df2[['dfore', 'dback']].rename(columns={"dfore":"dfore2", "dback":"dback2"})
d0 = pd.concat((df, m2), axis=1)
ofeature = ["dfore", "dback", "dfore2", "dback2"]

In [4]:
# d0[ofeature].to_hdf("shared.h5", "train", complevel=9)

In [5]:
c0 = pd.read_csv("/data/episode2/miha_train_add.csv", index_col=0, usecols=["index", "grid_value", "smoothgrid_value"])

/opt/conda/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
ct0 = pd.read_csv("/data/episode2/miha_test_add.csv")

In [7]:
d0 = pd.concat([d0, c0], axis=1)

In [8]:
d0.head()

,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number,X1,Y1,X2,Y2,dfore,dback,dfore2,dback2,grid_value,smoothgrid_value
index,,,,,,,,,,,,,,,,,,,
0,35130,50550.007812,66924.796875,65943.0,0.639179,-0.086816,0.583319,1.0,53,51376.465940,66812.543260,52202.924067,66700.289645,1.585658e+05,4.217776e+05,1.452454e+06,9.732423e+05,1.257399,4.538903
1,-999,48811.390625,65584.648438,12930.0,0.569996,0.294196,1.582101,0.0,49,49548.395837,65965.044417,50285.401050,66345.440397,1.121876e+06,1.481368e+05,3.367042e+06,3.826505e+06,1.252355,4.327241
2,-999,30120.525391,28329.425781,21981.0,-0.366648,-0.379520,1.292502,0.0,39,29646.449426,27838.706307,29172.373461,27347.986833,1.072219e+05,1.042209e+06,1.790224e+07,1.768661e+06,0.830752,3.235831
3,-999,67186.773438,23033.931641,68529.0,-0.434561,0.502130,2.832125,0.0,65,66624.886046,23683.185540,66062.998655,24332.439440,1.063053e+06,2.259203e+06,3.716566e+05,1.625907e+06,1.148058,3.847546
4,-999,64832.343750,64279.328125,7758.0,0.139993,-0.417299,2.959314,0.0,13,65013.354136,63739.760361,65194.364523,63200.192596,1.009342e+06,2.250782e+06,2.417186e+06,3.035434e+06,0.953308,4.033998


In [9]:
ef = pd.read_hdf("data-r1.h5", "test")
ef2 = pd.read_hdf("data-r2.h5", "test")

e2 = ef2[['dfore', 'dback']].rename(columns={"dfore":"dfore2", "dback":"dback2"})
e0 = pd.concat((ef, e2), axis=1)

In [10]:
# e0[ofeature].to_hdf("shared.h5", "test", complevel=9)

In [11]:
features = ["X", "Y", "Z", "TX", "TY", "dfore", "dback", "dfore2", "dback2", "chi2", "grid_value", "smoothgrid_value"]
features2 =  ["X", "Y", "Z", "TX", "TY", "chi2"]
tb = [0, 1, 2, 9, 14, 32, 40, 55, 75, 79, 80, 81, 82, 85, 93]
test = d0.loc[d0.brick_number.isin(tb)]
train = d0.loc[~d0.brick_number.isin(tb)]

In [ ]:
%%time
xgb2 = xgboost.XGBClassifier(n_estimators=500, nthread=20, max_depth=4, reg_alpha=0, reg_lambda=0.5)
xgb2.fit(train[features2], train.signal)

CPU times: user 13h 6min 58s, sys: 2h 25min 19s, total: 15h 32min 17s
Wall time: 1h 13s


In [ ]:
xgb1 = xgboost.XGBClassifier(n_estimators=500, nthread=20, max_depth=4, reg_alpha=0, reg_lambda=0.5)
xgb1.fit(train[features], train.signal)

In [ ]:
print(roc_auc_score(test.signal, xgb1.predict_proba(test[features])[:,1]))